In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

In [ ]:
import argparse
import gc
import os
import pickle as pkl
import random
import sys
import time
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from tqdm import tqdm

sys.path.append("../util")
from feature_engineering import lag_feature, log_feature, plot_loghist, count_in_session, first_in_session
from utils import reduce_df, save_checkpoint, seed_torch, train_epoch, val_epoch, mrr_at_k

#input_path = Path('D:/SIGIR/train/')
input_path = Path('/content/drive/MyDrive/Data_Competitions/SIGIR2021/')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
SessionId = 'SessionId'

# load

In [ ]:
%%time
all_df = pd.read_csv(input_path / 'product_train_and_test_4.csv')
print(all_df.shape)

In [ ]:
all_df

In [ ]:
all_df['num_following_pageview'] = all_df['num_following_pageview'].astype(np.int8)
all_df['num_following_search'] = all_df['num_following_search'].astype(np.int8)

In [ ]:
all_df.dtypes

# meta

In [ ]:
meta_df = pd.read_csv(input_path / 'meta_df.csv')

In [ ]:
meta_df

In [ ]:
meta_cols = ['product_sku_hash', 'category_hash', 'price_bucket', 'price_null']
meta_df = meta_df[meta_cols]

In [ ]:
meta_df['price_bucket'] = meta_df['price_bucket'].astype(np.int8)
meta_df

In [ ]:
all_df = pd.merge(all_df, meta_df, how='left', on='product_sku_hash')

In [ ]:
all_df

In [ ]:
gc.collect()

In [ ]:
all_df['price_bucket'].hist()

In [ ]:
all_df['price_bucket'].isnull().sum()

In [ ]:
all_df['category_hash'].value_counts()

In [ ]:
all_df['category_hash'].isnull().sum()

### target count

In [ ]:
tmp_df = all_df.groupby('product_sku_hash').SessionId.count().reset_index()
tmp_df.columns = ['product_sku_hash', 'product_count']
all_df = all_df.merge(tmp_df, how='left', on='product_sku_hash')

In [ ]:
gc.collect()

In [ ]:
all_df['product_sku_hash'].nunique()

In [ ]:
all_df['hashed_url'].nunique()

In [ ]:
all_df['product_count'].value_counts().sort_index()

In [ ]:
c = 'product_count'
_ = plt.hist(all_df[c], bins=100)
all_df[c].mean(), all_df[c].std()

In [ ]:
plot_loghist(all_df['product_count'], 100)

In [ ]:
del tmp_df
gc.collect()

In [ ]:
count_col = 'hashed_url'
tmp_df = all_df.groupby(count_col).SessionId.count().reset_index()
tmp_df.columns = [count_col, count_col + '_count']
all_df = all_df.merge(tmp_df, how='left', on=count_col)
del tmp_df

### low number targets

In [ ]:
all_df['hashed_url_count'].value_counts().sort_index()

In [ ]:
LOW_TARGETS_THR = 7

In [ ]:
all_df.loc[all_df.product_count <= LOW_TARGETS_THR, 'product_sku_hash'] = -1

In [ ]:
LOW_URLS_THR = 7
c = 'hashed_url'
all_df.loc[all_df[c + '_count'] <= LOW_URLS_THR, c] = -1

In [ ]:
all_df

In [ ]:
all_df[all_df['hashed_url'] == -1]

In [ ]:
gc.collect()

In [ ]:
all_df.dtypes

### time

In [ ]:
%%time

all_df['time'] = (all_df['server_timestamp_epoch_ms'] / 1000).astype(int)
all_df['time'] = pd.to_datetime(all_df['time'].astype(int), unit='s')

In [ ]:
all_df

In [ ]:
all_df['dayofweek'] = all_df.time.dt.weekday
all_df['hour'] = all_df.time.dt.hour

In [ ]:
all_df['weekend'] = all_df['dayofweek'].isin([5, 6]).astype('int8')

In [ ]:
_ = plt.hist(all_df['hour'], 24)

In [ ]:
_ = plt.hist(all_df['dayofweek'], 7)

In [ ]:
gc.collect()

### event counters

In [ ]:
_ = plt.hist(all_df['cum_product'], bins=100)

In [ ]:
_ = plt.hist(all_df['cum_product_r'], bins=100)

In [ ]:
(all_df['cum_product'] == 0).sum()

In [ ]:
all_df['cum_search_last'] = lag_feature(all_df, SessionId, 'cum_search', 1, 0).astype('int32')

In [ ]:
all_df[all_df['cum_search'] > 0][['cum_search', 'cum_search_last']]

In [ ]:
all_df['num_search'] = all_df['cum_search'] - all_df['cum_search_last']

In [ ]:
_ = plt.hist(all_df['num_search'], bins=100)

In [ ]:
_ = plt.hist(all_df['cum_pageview'], bins=100)

In [ ]:
all_df['cum_pageview_last'] = lag_feature(all_df, SessionId, 'cum_pageview', 1, 0).astype('int32')
all_df[all_df['cum_pageview'] > 0][['cum_pageview', 'cum_pageview_last']]

In [ ]:
all_df['num_pageview'] = all_df['cum_pageview'] - all_df['cum_pageview_last']

In [ ]:
_ = plt.hist(all_df['num_pageview'], bins=100)

In [ ]:
all_df

In [ ]:
c = 'cum_event'
_ = plt.hist(all_df[c], bins=100)

In [ ]:
c = 'num_following_search'
_ = plt.hist(all_df[c], bins=100)

In [ ]:
c = 'num_following_pageview'
_ = plt.hist(all_df[c], bins=100)

In [ ]:
c = 'cum_search_last'
_ = plt.hist(all_df[c], bins=100)

In [ ]:
c = 'cum_pageview_last'
_ = plt.hist(all_df[c], bins=100)

# Encoding

In [ ]:
all_df.columns

In [ ]:
CATS = ['product_sku_hash', 'category_hash', 'product_action',
        'cum_pageview', 'cum_product', 'cum_search', 'cum_event',
        'num_following_search', 'num_following_pageview',
        'price_bucket', 'price_null',
        'dayofweek', 'hour', 'weekend',
        'num_search', 'num_pageview', 'hashed_url']
mappings = []
for c in CATS:
    all_df[c+'_id'], mp = all_df[c].factorize()
    mappings.append(mp)

In [ ]:
all_df

In [ ]:
all_df[all_df['product_sku_hash'] == -1]

In [ ]:
LOW_TARGETS = np.where(mappings[0] == -1)[0][0]
LOW_TARGETS

In [ ]:
mappings

In [ ]:
with open(input_path / 'mappings_rnn_stage2.pkl', mode='wb') as f:
    pkl.dump(mappings, f)

In [ ]:
NUM_TARGETS = all_df.product_sku_hash_id.max() + 1
NUM_TARGETS

# Reload

In [ ]:
%%time
all_df.to_csv(input_path / 'inputdf_meta_rnn_stage2_pre.csv', index=False)

In [ ]:
%%time
all_df = pd.read_csv(input_path / 'inputdf_meta_rnn_stage2_pre.csv')

### lag feartures

In [ ]:
LAGS = 19
lag_targets = []

for i in range(1, LAGS+1):
    all_df[f'product_hash_lag{i}'] = lag_feature(all_df, SessionId, 'product_sku_hash_id', i, NUM_TARGETS).astype('int32')
    lag_targets.append(f'product_hash_lag{i}')

In [ ]:
lag_targets

In [ ]:
all_df[lag_targets]

In [ ]:
for c in lag_targets:
    print((all_df[c] == NUM_TARGETS).sum())

In [ ]:
NUM_CATEGORIES = all_df.category_hash_id.max() + 1
NUM_CATEGORIES

In [ ]:
LAGS_CATEGORY = 19
lag_categories = []

for i in range(1, LAGS_CATEGORY+1):
    all_df[f'category_lag{i}'] = lag_feature(all_df, SessionId, 'category_hash_id', i, NUM_CATEGORIES).astype('int32')
    lag_categories.append(f'category_lag{i}')

In [ ]:
NUM_URLS = all_df.hashed_url_id.max() + 1
NUM_URLS

In [ ]:
LAGS_URL = 19
lag_urls = []

for i in range(1, LAGS_URL+1):
    all_df[f'url_lag{i}'] = lag_feature(all_df, SessionId, 'hashed_url_id', i, NUM_URLS).astype('int32')
    lag_urls.append(f'url_lag{i}')

In [ ]:
lag_urls

### first target and category for each session

In [ ]:
%%time
tmp_df = all_df[all_df['cum_product']==1][[SessionId, 'product_sku_hash_id']]
tmp_df.columns = [SessionId, 'first_product']
all_df = all_df.merge(tmp_df, on=SessionId, how='left')
del tmp_df

In [ ]:
%%time
tmp_df = all_df[all_df['cum_product']==1][[SessionId, 'category_hash_id']]
tmp_df.columns = [SessionId, 'first_category']
all_df = all_df.merge(tmp_df, on=SessionId, how='left')
del tmp_df

In [ ]:
%%time
tmp_df = all_df[all_df['cum_product']==1][[SessionId, 'hashed_url_id']]
tmp_df.columns = [SessionId, 'first_url']
all_df = all_df.merge(tmp_df, on=SessionId, how='left')
del tmp_df

In [ ]:
%%time
all_df.to_csv(input_path / 'inputdf_meta_rnn_stage2_pre2.csv', index=False)

In [ ]:
gc.collect()

In [ ]:
all_df

In [ ]:
LOW_TARGETS = 6

### targets

In [ ]:
all_df = all_df.rename(columns={'product_sku_hash_id': 'product_hash_lag0'})

In [ ]:
all_df['next_product'] = lag_feature(all_df, SessionId, 'product_hash_lag0', -1, NUM_TARGETS).astype('int32')

In [ ]:
((all_df.cum_product_r > 0) & (all_df['next_product'] == LOW_TARGETS)).sum()

In [ ]:
((all_df.cum_product_r > 0) & (all_df['next_product'] == NUM_TARGETS)).sum()

In [ ]:
all_df['next_next_product'] = lag_feature(all_df, SessionId, 'next_product', -1, LOW_TARGETS).astype('int32')

In [ ]:
all_df.loc[all_df['next_product'] == LOW_TARGETS, 'next_product'] = all_df.loc[all_df['next_product'] == LOW_TARGETS, 'next_next_product']

In [ ]:
gc.collect()

In [ ]:
((all_df.cum_product_r > 0) & (all_df['next_product'] == LOW_TARGETS)).sum()

In [ ]:
((all_df.cum_product_r > 0) & (all_df['next_product'] == NUM_TARGETS)).sum()

In [ ]:
all_df.loc[all_df['next_product'] == NUM_TARGETS, 'next_product'] = LOW_TARGETS

In [ ]:
lag_targets += ['product_hash_lag0']

In [ ]:
lag_targets

In [ ]:
all_df[lag_targets + ['next_product']]

In [ ]:
for c in all_df.columns:
    print(c)

In [ ]:
gc.collect()

# fold

In [ ]:
N_FOLD = 5

all_df['fold'] = 0
group_kfold = GroupKFold(n_splits=N_FOLD)
for fold, (train_idx, test_idx) in enumerate(group_kfold.split(X=all_df, groups=all_df[SessionId])):
    all_df.iloc[test_idx, len(all_df.columns)-1] = fold

all_df['fold'].value_counts()

In [ ]:
all_df.columns

### unnecessary cols

### cols to remove
product\_action, product\_sku\_hash, hashed\_url, server\_timestamp\_epoch\_ms, session\_id2, 
time, next_event_time, first_time, last_time

### cols not to use
SessionId, sorting,

### cols to use
istest, fold, product\_count, product_sku_hash_id, product_hash_lag{k}, first_product, dayofweek, hour, lapse, event_length, weekend, event_count, event_count_r, add, purchase, remove, next_product

In [ ]:
cols_to_remove = ['product_action', 'product_sku_hash', 'server_timestamp_epoch_ms',
                  'hashed_url', 'last_event_length',
                  'cum_pageview', 'cum_product', 'cum_search', 'cum_event', 'lapse',
                  'num_following_search', 'num_following_pageview',
                  'category_hash', 'price_bucket', 'price_null',
                  'product_count', 'time', 'dayofweek', 'hour', 'weekend',
                  'cum_search_last', 'num_search', 'cum_pageview_last', 'num_pageview', 'next_next_product',
                  'same_url', 'last_hashed_url']

In [ ]:
for c in cols_to_remove:
    if c in all_df.columns:
        del all_df[c]

In [ ]:
gc.collect()

In [ ]:
all_df

In [ ]:
all_df.columns

### check nulls

In [ ]:
all_df.isnull().sum().sum()

In [ ]:
gc.collect()

# export

In [ ]:
all_df.dtypes

In [ ]:
reduce_df(all_df)
all_df.dtypes

In [ ]:
all_df

In [ ]:
%%time
all_df.to_csv(input_path / 'inputdf_meta_rnn_stage2.csv', index=False)

# That's pretty much about it. Move to the next notebook!